# Feature Engineering
Explanation:

## Import Packages

In [17]:
from importnb import Notebook
import pandas as pd
import numpy as np

## Dataframe import from 'DataPreprocessing'

In [22]:
with Notebook():
    from DataPreprocessing_Pipeline import companies, funding, investors

## New Features: Companies <> Investment Rounds
Explanation:

### Feature 1: Months Until First Round
Number of months between the funding date and the first investment round. Explanation: The table 'all_rounds' has every investment round made during the existence of the companies. We already mapped the respectively funding round to the startup. 

In [24]:
def calculate_months_until_first_round(company_id, founded_date):
    rounds = funding[funding['Company ID'] == company_id]

    if rounds.empty:
        return np.nan

    first_round_date = rounds['Announced Date'].min()

    delta_months = (first_round_date.year - founded_date.year) * 12 + (first_round_date.month - founded_date.month)
    return max(delta_months, 0)

companies['Months until First Round'] = companies.apply(
    lambda row: calculate_months_until_first_round(row['ID'], row['Founded Date']),
    axis=1
)

### Feature 2: 

## New Features: Companies <> Investors
Explanation:

### Feature 1: 

## Feature 2: 

## New Features: Companies